In [15]:
import re
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
import pandas as pd

In [ ]:
links = []
for i in range(36):
    links.append('https://zona.media/_load?selector=onlines&page={}&total=887&excludeIds=51745%2C51314%2C51201%2C51154%2C50720&type=&_=1634045363911'.format(i))

In [ ]:
links[0]

'https://zona.media/_load?selector=onlines&page=0&total=887&excludeIds=51745%2C51314%2C51201%2C51154%2C50720&type=&_=1634045363911'

In [ ]:
articles = []
for l in tqdm(links):
    page = requests.get(l)
    soup = BeautifulSoup(page.content, features = 'html.parser')
    html = str(soup)
    arts = re.findall('inner.{50}href.{4}(.+?).{1}"', html)
    for a in arts:
        articles.append('https://zona.media{}'.format(a))
    

100%|██████████| 36/36 [00:31<00:00,  1.15it/s]


In [ ]:
len(articles)

710

In [ ]:
articles[:10]

['https://zona.media//online/2021/04/29/navalny-appeal',
 'https://zona.media//online/2021/04/22/navalny-after',
 'https://zona.media//online/2021/04/21/navalny-msk-spb',
 'https://zona.media//online/2021/04/21/all-russia',
 'https://zona.media//online/2021/04/14/doxa',
 'https://zona.media//online/2021/04/14/sobol-5',
 'https://zona.media//online/2021/04/13/sobol-3',
 'https://zona.media//online/2021/04/12/sobol-3',
 'https://zona.media//online/2021/03/23/lomov23',
 'https://zona.media//online/2021/03/22/golunov14']

In [ ]:
texts = []
titles = []
dates = []
sources = []

for url in tqdm(articles):
    try:
        page = requests.get(url)
        soup = BeautifulSoup(page.content, features = 'html.parser')
    
        try:
            soup.figure.decompose()
        except AttributeError:
            pass
    
        title = soup.find('header', class_='mz-publish__title').getText()
        date = soup.find_all('div', class_='mz-publish-meta__item')[1].getText().split(',')[0]
        statements = soup.find_all('body')[1:]
        titles.append(title)
        dates.append(date)
        sources.append(url)
        sents = []
        
        for statement in statements:
            sent = statement.getText()
            sents.append(sent.strip())
        
        texts.append(" ".join(sents))
    
    except ConnectionError as err:
        print(err)

100%|██████████| 710/710 [16:49<00:00,  1.42s/it]  


In [ ]:
print(len(titles), len(texts), len(dates), len(sources))

710 710 710 710


In [ ]:
df = pd.DataFrame({
    'title': titles,
    'text': texts,
    'date': dates,
    'source': sources
})

In [ ]:
df.head(10)

,title,text,date,source
0,Навальный и дело о клевете на ветерана. Апелляция,В середине июня 2020 года Следственный комитет...,29 апреля 2021,https://zona.media/online/2021/04/29/navalny-a...
1,«Свободу Навальному». После протестов,«ОВД-Инфо» обновил списки задержанных на акция...,22 апреля 2021,https://zona.media/online/2021/04/22/navalny-a...
2,«Свободу Навальному». Акции в Москве и Петербурге,Первоначально соратники Алексея Навального соб...,21 апреля 2021,https://zona.media/online/2021/04/21/navalny-m...
3,«Свободу Навальному». Протесты по всей России,"По данным сайта «Свободу Навальному», акции пр...",21 апреля 2021,https://zona.media/online/2021/04/21/all-russia
4,Дело о митингах и подростках. Мера пресечения ...,Утром 14 апреля силовики пришли с обысками к ж...,14 апреля 2021,https://zona.media/online/2021/04/14/doxa
5,«Квартирное дело». Соболь в суде. День пятый,Предыдущее заседание в Перовском суде началось...,14 апреля 2021,https://zona.media/online/2021/04/14/sobol-5
6,«Квартирное дело». Соболь в суде. День четвертый,На прошлом заседании Любовь Соболь все же допу...,13 апреля 2021,https://zona.media/online/2021/04/13/sobol-3
7,«Квартирное дело». Соболь в суде. День третий,В Перовском районном суде продолжается выездно...,12 апреля 2021,https://zona.media/online/2021/04/12/sobol-3
8,Геодезист Ломов. Условный срок за акцию протес...,39-летнего инженера лесного хозяйства из Петер...,23 марта 2021,https://zona.media/online/2021/03/23/lomov23
9,Суд над полицейскими по делу Голунова. День 14,На заседании 18 марта суд завершил в закрытом ...,22 марта 2021,https://zona.media/online/2021/03/22/golunov14


In [ ]:
df.to_csv('/Users/zoiabutenko/PycharmProjects/3rdcourse/project/corpus.csv')

In [4]:
import sqlite3

In [5]:
conn = sqlite3.connect('corpus.db')  # подключение
cur = conn.cursor()  # курсор

In [1]:
import stanza

In [2]:
stanza.download('ru')

2021-10-15 15:09:46 INFO: Downloading default packages for language: ru (Russian)...
2021-10-15 15:09:48 INFO: File exists: /Users/zoiabutenko/stanza_resources/ru/default.zip.
2021-10-15 15:09:56 INFO: Finished downloading models and saved to /Users/zoiabutenko/stanza_resources.


In [19]:
nlp = stanza.Pipeline(lang='RU', processors='tokenize,pos,lemma')

2021-10-12 20:53:49 INFO: Loading these models for language: ru (Russian):
| Processor | Package   |
-------------------------
| tokenize  | syntagrus |
| pos       | syntagrus |
| lemma     | syntagrus |

2021-10-12 20:53:49 INFO: Use device: cpu
2021-10-12 20:53:49 INFO: Loading: tokenize
2021-10-12 20:53:49 INFO: Loading: pos
2021-10-12 20:53:49 INFO: Loading: lemma
2021-10-12 20:53:50 INFO: Done loading processors!


In [20]:
def parse_text(text_):
  doc = nlp(text_)
  stanza_tagged = []
  for sent in range(len(doc.sentences)):
      for word in range(len(doc.sentences[sent].words)):
          stanza_tagged.append((doc.sentences[sent].words[word].text, 
                                doc.sentences[sent].words[word].lemma, 
                                doc.sentences[sent].words[word].upos,
                                word, sent))
  return stanza_tagged

In [12]:
query = '''
SELECT * FROM corpus
'''

df = pd.read_sql_query(query, con=conn)

In [13]:
df.head(3)

,index,title,text,date,source
0,0,Навальный и дело о клевете на ветерана. Апелляция,В середине июня 2020 года Следственный комитет...,29 апреля 2021,https://zona.media/online/2021/04/29/navalny-a...
1,1,«Свободу Навальному». После протестов,«ОВД-Инфо» обновил списки задержанных на акция...,22 апреля 2021,https://zona.media/online/2021/04/22/navalny-a...
2,2,«Свободу Навальному». Акции в Москве и Петербурге,Первоначально соратники Алексея Навального соб...,21 апреля 2021,https://zona.media/online/2021/04/21/navalny-m...


In [21]:
indexes = []
ord_words = []
sentence_nums = []
tokens = []
lemmas = []
poses = []

for i in tqdm(range(len(list(df['text'])))):
  tagged = parse_text(list(df['text'])[i])
  for t in tagged:
    indexes.append(i)
    ord_words.append(t[3])
    sentence_nums.append(t[4])
    tokens.append(t[0])
    lemmas.append(t[1])
    poses.append(t[2])

100%|██████████| 710/710 [1:23:48<00:00,  7.08s/it]


In [17]:
len(indexes), len(ord_words), len(sentence_nums), len(tokens), len(lemmas), len(poses)

(446909, 446909, 446909, 446909, 446909, 446909)

In [22]:
df_tokens = pd.DataFrame({
    'index_id': indexes,
    'word_num': ord_words,
    'sentence_num': sentence_nums,
    'token': tokens,
    'lemma': lemmas,
    'pos': poses
})

In [23]:
df_tokens.sample(10)

,index_id,word_num,sentence_num,token,lemma,pos
55278,16,10,145,документ,документ,NOUN
1522228,438,7,74,рядом,ряд,NOUN
659400,195,5,39,позвонила,звонить,VERB
2311470,641,14,266,присяжных,присяжный,NOUN
2480989,679,1,47,В,в,ADP
1782138,515,19,13,инцидента,инцидент,NOUN
1342828,392,5,162,Собянин,Собянин,PROPN
1348512,395,15,13,",",",",PUNCT
862304,244,1,316,узнает,узнавать,VERB
2527782,704,74,21,идеологической,идеологический,ADJ


In [24]:
df_tokens.to_sql('tokens', con=conn)

In [25]:
import os
os.path.getsize('corpus.db')

174194688